In [1]:
import json
import requests
import numpy as np
import pandas as pd

In [27]:
# TOP collections in OpenSea marketplace

top_collections = {
    "Bored Ape Yacht Club": "0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D",
    "CryptoPunks": "0xb47e3cd837dDF8e4c57F05d70Ab865de6e193BBB",
    "Mutant Ape Yacht Club": "0x60E4d786628Fea6478F785A6d7e704777c86a7c6",
    "Mutant Hound Collars": "0xaE99a698156eE8F8d07Cbe7F271c31EeaaC07087",
    "Yaypegs": "0x7fda36c8daEDcc55B73E964c2831D6161eF60a75",
    "Terraforms by Mathcastles": "0x4E1f41613c9084FdB9E34E11fAE9412427480e56",
    "Otherdeed for Otherside": "0x34d85c9CDeB23FA97cb08333b511ac86E1C4E258",
    "Wolf Game": "0x7F36182DeE28c45dE6072a34D29855BaE76DBe2f",
    "Bored Ape Kennel Club": "0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623",
    "Valhalla": "0x231d3559aa848Bf10366fB9868590F01d34bF240",
    "oraand": "0xF21Ab54111EB5049e42f4794D2724658Baa42FC1",
    "Savage Nation": "0x61E3D1C26802DF805e9Fc22Dc26342e29eCFe860",
    "Chimera King": "0x6950f7Ec392911De504A79F5334D39F4933fAF25",
    "Wired Beast": "0x2910312A1e3e4cdf3c33dFBc88d1e1d7a22e2Bbf",
    "Project SHINOBIS": "0xc64efC58fb656f389F0F7C25C92B7ecF8D02D740"
}

In [3]:
def run_query(query: str)->dict:
    headers = {'X-API-KEY': 'BQYVMR0gpKfJf8Ua04OP21jJ2VVVOjzu'}
    request = requests.post('https://graphql.bitquery.io/',
                            json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}.      {}'.format(request.status_code,
                        query))

In [34]:
def query_func(address: str)->str:
    query = """
    query MyQuery 
    {
    ethereum {
        transfers(currency: {is: \"""" + f"{address}" + """\" },
        options: {desc: "amount", limit: 100, offset: 0}){
            sender {
                address
            }
            amount
            count
            receiver_count: count(uniq: receivers)
            max_amount: maximum(of: amount, get: amount)
            max_date: maximum(of: date)
            receiver {
                address
            }
            }
        }
    }
    """
    return query

In [36]:
def parser():
    unique_sender_address = set()
    wallet_collection = {}

    for collection, address in top_collections.items():
        query = query_func(address)
        result = run_query(query)
        infos = result['data']['ethereum']['transfers']
        if type(infos) != list:
            print(collection, address)
        else:
            for i in range(len(infos)):
                if infos[i]['receiver']['address'] not in unique_sender_address:
                    unique_sender_address.add(infos[i]['receiver']['address'])
                    wallet_collection[infos[i]['receiver']['address']] = []
                curr_wallet_address = infos[i]['receiver']['address']
                wallet_collection[curr_wallet_address].append(collection)
    
    return unique_sender_address, wallet_collection

In [37]:
def matrix(unique_sender_address, wallet_collection):
    # Create df with NaN values
    df = pd.DataFrame(columns=top_collections.keys())
    df['wallet_address'] = list(unique_sender_address)
    df.set_index('wallet_address')
    # Get numpy array of df
    data = df.to_numpy()
    # Fill matrix
    for i in range(data.shape[0]):
        address = data[i][-1]
        owned_nfts = wallet_collection[address]
        for owned_nft in owned_nfts:
            if owned_nft in df.columns:
                j = df.columns.get_loc(owned_nft)
                data[i][j] = 1
            
    df = pd.DataFrame(data, columns=df.columns)
    df.fillna(0, inplace=True)
    df.set_index('wallet_address', inplace=True)
    return df

In [38]:
unique_sender_address, wallet_collection = parser()

In [39]:
df = matrix(unique_sender_address, wallet_collection)

In [40]:
df.to_csv('result.csv')

In [41]:
from xgboost import XGBClassifier
import pickle

df = pd.read_csv('result.csv', index_col=0)
for i in df.columns:    
    X = df.drop(i, axis=1)
    y = df.loc[:,i]
        
    model = XGBClassifier()
    model.fit(X, y.values.ravel()) 

    
    pickle.dump(model, open(f'models/{i}.pkl', 'wb'))

C:\Users\baimy\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\baimy\miniconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\baimy\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False wh

[21:36:15] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:36:15] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\baimy\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\baimy\miniconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\baimy\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False wh

[21:36:15] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:36:15] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\baimy\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\baimy\miniconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\baimy\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False wh

...

C:\Users\baimy\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\baimy\miniconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\baimy\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False wh

[21:36:16] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:36:16] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\baimy\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\baimy\miniconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\baimy\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False wh

[21:36:16] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:36:17] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:36:17] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\baimy\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\baimy\miniconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [42]:
df

,Bored Ape Yacht Club,CryptoPunks,Mutant Ape Yacht Club,Mutant Hound Collars,Yaypegs,Terraforms by Mathcastles,Otherdeed for Otherside,Wolf Game,Bored Ape Kennel Club,Valhalla,oraand,Savage Nation,Chimera King,Wired Beast,Project SHINOBIS
wallet_address,,,,,,,,,,,,,,,
0xbb372b06aaeef2a869d4519c92658a2a6a493c07,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0x3adc65bb7ecfd4482ae6fc9a7d5b4dcc7e248263,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0x9d1cb67fd65fa0b842a873ebdc269f6f71988e9e,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0x6266dbb2d202d4e246ee86d76bb2fbb9a71eafcd,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0x4897d38b0974051d8fa34364e37a5993f4a966a5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0x5a418d8bc0c074a4a8fa88d1322dc51cc1cb9d29,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0x8c5c9d500a5ce9322b28ed5c6dbcaa16c8ea4eaa,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0x37556464d037536a0603fc824073d34e9893c880,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
